# ***Machine Learning Prueba 2 - Analizando los crímenes en la Ciudad de Nueva York***.
### Nombre(s): Thomas Peet, Braulio Águila, Camilo Ramírez
### Generación: G47
### Profesores: Alfonso Tobar - Sebastián Ulloa
### Fecha: 06-10-2022

### *Contexto*
En esta ocasión trabajaremos con datos públicos del departamento de policía de New York.    
El dataset es llamado stop_and_frisk_data y contiene información sobre interrogaciones    
y detenciones realizadas por el departamento de policía de NY en la vía pública. El    
diccionario de atributos se encuentra en el archivo 2009 SQF File Spec.xls.    
Para todo nuestro estudio utilizaremos los datos correspondientes al año 2009 como    
conjunto de entrenamiento y los datos del 2010 como conjunto de pruebas. Hay que hacer    
notar que los datos que estamos utilizando son un muestreo del de la cantidad de registros    
reales que contiene el dataset, esta decisión fue tomada debido a los largos tiempos de    
entrenamiento y procesamiento que requiere el volumen de datos reales. 

### *Objetivos*
Para alcanzar el objetivo general, su trabajo se puede desagregar en los siguientes puntos:  

1. Debe analizar de forma exploratoria los atributos. Reporte la cantidad de datos    
perdidos y presente su esquema de recodificación. Tenga presente que lo que    
observe en el análisis exploratorio debe guiar su proceso de ingeniería de atributos,    
por lo que se le recomienda que piense en aspectos de las variables involucradas     
que puedan afectar el proceso mencionado.

2. Reporte la probabilidad de que un individuo sea arrestado en uno de los cinco  
barrios, condicional al género y a la raza. Concluya, ¿qué implicancias éticas tienen  
algunas conclusiones de lo que observa?.

3. Entregue un modelo predictivo que prediga efectivamente si un determinado  
procedimiento concluirá en un arresto o no. Para ello, guíate por los siguientes  
lineamientos:  
    - Entrene por lo menos 3 modelos que sean capaces de predecir si se  
producirá un arresto o no. Una vez que encuentre un modelo satisfactorio,  
reporte al menos dos métricas de desempeño.  
    - Refine aquellos atributos relevantes con alguna estrategia que crea  
conveniente y reporte por lo menos 5 atributos relevantes para realizar la  
predicción.

4. Genere al menos cinco modelos predictivos que permitan determinar si el  
procedimiento policial concluirá en alguna acción violenta.  
○ Para ello, debe generar un nuevo atributo como vector objetivo que indique  
cuándo hubo violencia o no. Éste debe ser creado a partir de atributos  
existentes que indiquen el tipo de violencia.
  
5. Seleccione los 2 mejores modelos, serialicelos y envíalos a evaluación. Recuerde que  
el modelo serializado debe ser posterior al fit, para poder ejecutar predict en los  
nuevos datos

>>>#### Tipo de problema a resolver:
- De acuerdo con el enunciado y una revisión preliminar de los datos entregados, ambas problemáticas planteadas,  
el hecho de que ocurra o no un arresto, y de que un procedimiento policial es o no violento, corresponden a   
problemas de **clasificación**, ya que ambas variables objetivos son discretas.

>>>#### Tipo de métricas a implementar:
- Las métricas que se utilizarán para la división de muestras corresponden a :
- Tipo de preprocesamiento: 

>>>#### Modelos (5) con gridsearch e hiperparamétros tentativos/definitivos:
- Modelo 1:
- Modelo 2:
- Modelo 3:
- Modelo 4:
- Modelo 5:

>>>#### Comportamiento de variables objetivo (recodificados):
- Variable objetivo 1: procedimiento policial en el que ocurre o no un arresto ("arstmade")
- Variable objetivo 2: procedimiento policial 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline

from feature_engine.imputation import CategoricalImputer, MeanMedianImputer
from feature_engine.encoding import OrdinalEncoder, OneHotEncoder
from feature_engine.wrappers import SklearnTransformerWrapper
#from feature engine drop features ----> sacar features para el pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

from sklearn import set_config
set_config(display='diagram')

from pathlib import Path
from pandas_profiling import ProfileReport

import geopandas as gpd
import contextily as cx
#import shapely
#import folium 
import pyproj
import helpers as hp
import preproc_nyc_sqf_V2 as preproc
import contextily as cx
#pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

## ***Dataset interrogaciones/detenciones Policía de Nueva York - 2009***:

## Preprocesamiento del dataframe

In [2]:
from sklearn.preprocessing import FunctionTransformer
import sys
sys.path.append('..')

from utils import CreateSuitableDataframeTransformer
from utils import OrdinalEncoderFixedTransformer
from utils import DropRowsTransformer
from utils import CriterioExperto
import utils

## Definición de los datasets para los 2 problemas de clasificación

In [3]:
# Considerar variables relevantes para modelos de procedimiento que concluye en arresto y/o procedimiento que concluye en situación violenta:
# redefinir variable "others" (esta mál escrita con respecto a la planilla)

mantener = ["var_pf", "rf_vcrim", "rf_othsw	", "cs_descr", "arstmade", "contrabn",
"pistol", "riflshot", "asltweap", "knifcuti", "othrweap", "rf_vcact", "ac_evasv", "offshld", "cs_drgtr","sb_outln",	"sb_hdobj", "ac_time", "rf_knowl", "ac_assoc", "rf_rfcmp", "cs_vcrim", "ac_incid", "rf_verbl", "sex", "city", "race", "month", "xcoord", "ycoord", "sb_admis"]

var_eliminar_pf = ["pf_baton", "pf_hcuff", "pf_pepsp", "pf_other", "pf_ptwep", "pf_drwep", "pf_wall", "pf_hands", "pf_grnd"] #9 variables eliminadas

var_eliminar_others = ["ac_rept", "ac_inves", "ac_proxm", "cs_casng", "cs_lkout", "explnstp", 
"sumissue", "offunif", "officrid", "frisked", "searched", "cs_cloth", "offverb", "rf_furt",
"sb_other", "ac_other", "rf_bulg", "cs_furtv", "recstat", "cs_bulge", "cs_other", "trhsloc", "build", "beat", "post","rf_attir", "cs_objcs","eyecolor", "haircolr", "sector" ]

pendientes = ["radio", "inout", "othpers", "ac_stsnd", "ac_cgdir", "meters", "typeofid"]


In [4]:
df2009 = pd.read_csv('2009_1perc.csv', index_col=0)
x_1, y_1, x_2, y_2 = utils.split_features_target(df2009)
x_1.shape, y_1.shape, x_2.shape, y_2.shape

((5812, 110), (5812,), (5812, 102), (5812,))

* Se eliminan los datos sin xcoord o ycoord
* Se eliminan los registros de detenidos menores de 18 años

In [5]:
drt = DropRowsTransformer() # Creo que la mejor idea sería serializar esto y el pipeline central y mandarle los 2 a Alfonso
df2009_nonull = drt.transform(df2009)

x_1_i, y_1_i, x_2_i, y_2_i = utils.split_features_target(df2009_nonull)
print('El mismo dataframe aplicando los criterios de descartar datos sin xcoord/ycoord y registros de menores de 18 años')
x_1_i.shape, y_1_i.shape, x_2_i.shape, y_2_i.shape


El mismo dataframe aplicando los criterios de descartar datos sin xcoord/ycoord y registros de menores de 18 años


((4499, 111), (4499,), (4499, 103), (4499,))

In [6]:
df2009_nonull.city.mode()[0]

'BROOKLYN'

In [7]:
#Se utiliza función entregada para limpieza preliminar de la data, se obtiene un df nuevo procesado y dos listas:
# proc_df #este nuevo dataframe, estas constituido de 69 variables categóricas y 2 númericas sintéticas ("meters", "month")

csd = CreateSuitableDataframeTransformer()
preprocess = Pipeline(steps=[
    ('ce', CriterioExperto(columns = var_eliminar_others + var_eliminar_pf)),
    ('csd', csd),
    ('cat_imp', CategoricalImputer(imputation_method='frequent')),
    ('num_imp', MeanMedianImputer(imputation_method='mean')),
    ('oe', OrdinalEncoderFixedTransformer(encoding_method='ordered', csd=csd)),# aqui estoy pasando info desde el proceso csd (que son las columnas que seleccionamos como categoricas) para encodear.
    ('sc', SklearnTransformerWrapper(StandardScaler()))
])

# X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(x_1, y_1, test_size=.2, random_state=42)
# X_train_1_tr = preprocess.fit_transform(X_train_1, y_train_1)
# X_test_1_tr = preprocess.transform(X_test_1)
# X_train_1_tr
# X_test_1_tr

In [8]:
from sklearn.ensemble import StackingClassifier
from sklearn.naive_bayes import MultinomialNB

mnnb = MultinomialNB()
clf1 = GaussianNB()
clf2 = KNeighborsClassifier()
clf3 = SVC(random_state=42, probability=True)
clf4 = GradientBoostingClassifier(random_state=42)
meta0 = LogisticRegression(random_state=42) # funciona particularmente bien stackeando xgboost
meta1 = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10)
meta2 = GradientBoostingClassifier(random_state=42)

X_train, X_test, y_train, y_test = train_test_split(x_1, y_1, test_size=.2, random_state=42)

pipe = Pipeline(steps=[
    ('preprocessing', preprocess),
    ('model', meta1)
])

utils.train_function(pipe, X_train, X_test, y_train, y_test)

# search = GridSearchCV(pipe, params, cv=5, scoring='accuracy', n_jobs=-1)
# search.fit(X_train, y_train)
# search.best_params_, search.best_score_


train
              precision    recall  f1-score   support

           0     0.9627    1.0000    0.9810      4382
           1     1.0000    0.3633    0.5330       267

    accuracy                         0.9634      4649
   macro avg     0.9813    0.6816    0.7570      4649
weighted avg     0.9648    0.9634    0.9552      4649

test
              precision    recall  f1-score   support

           0     0.9448    0.9963    0.9699      1082
           1     0.8182    0.2222    0.3495        81

    accuracy                         0.9424      1163
   macro avg     0.8815    0.6093    0.6597      1163
weighted avg     0.9360    0.9424    0.9267      1163



Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('ce',
                                  CriterioExperto(columns=['ac_rept',
                                                           'ac_inves',
                                                           'ac_proxm',
                                                           'cs_casng',
                                                           'cs_lkout',
                                                           'explnstp',
                                                           'sumissue',
                                                           'offunif',
                                                           'officrid',
                                                           'frisked',
                                                           'searched',
                                                           'cs_cloth',
                                                           'offverb', 'rf_furt',
                                                           'sb_other',
                                                           'ac_other',
                                                           'rf_bulg',
                                                           'cs_furtv',
                                                           'recstat',
                                                           'cs_bulge',
                                                           'cs_other',
                                                           'trhsloc', 'build',
                                                           'beat', 'post',
                                                           'rf_attir',
                                                           'cs...
                                 ('cat_imp',
                                  CategoricalImputer(imputation_method='frequent')),
                                 ('num_imp',
                                  MeanMedianImputer(imputation_method='mean')),
                                 ('oe',
                                  OrdinalEncoderFixedTransformer(csd=CreateSuitableDataframeTransformer(),
                                                                 encoding_method='ordered')),
                                 ('sc',
                                  SklearnTransformerWrapper(transformer=StandardScaler()))])),
                ('model',
                 RandomForestClassifier(max_depth=10, random_state=42))])

>>>#### Exploración variable objetivo 1 ("arstmade")

In [9]:
utils.save_bytes_variable({'preprocess':preprocess, 'pipe':pipe, 'drt': drt}, 'crimenes.pickle')

In [ ]:
y_1.value_counts() #oversampling

In [ ]:
#Visualización de el vector objetivo:
sns.histplot(y_1, x="arstmade"); # Categoría uno muchos menos casos que categoría 0.
plt.title("Vector objetivo : arstmade")
plt.xlabel("clases");

>>>#### Exploración variable objetivo 2 ("violencia física en arresto") --- Necesita ser recodificada

In [ ]:
#Recodificando variable objetivo 2:

var_pf = proc_df.columns[np.where([i[0:2]=='pf' for i in proc_df.columns.tolist()])]. tolist()
u = proc_df[var_pf]
u["var_pf"] = [int(np.isin(["Y"], u.iloc[i].values.tolist())[0]) for i in range(0,len(u))]

proc_df.insert(proc_df.shape[1],"var_pf", u["var_pf"])
proc_df


In [ ]:
# hp.cat_num_rate_analysis(proc_df)

In [ ]:
proc_df.corr()

In [ ]:
#variable "sex" necesita ser recodificada (random choice?) para tener una variable categórica binaria en ejercicio de probabilidades.
#recodificar city?

In [ ]:
#Eliminar variables (Considerar drop_features)
#dropna(axis = 1, inplace= True)
#df_na = df1.dropna(how="all", axis=1)  
#dfnan = df1.dropna(subset=["ID"])

In [ ]:
#Formato para definir variables elegidas posterior al filtro

# # ***Listado de variables***:
# * `age`: Edad del individuo.
# * `workclass`: Naturaleza de la organización que emplea al individuo.
# * `education`: Nivel educacional del individuo:
#     | Variable          | Explicación                       |
#     | ----------------- | ------------------                |
#     | __Bachelors__     | (Licenciado)                      |
#     | __Some-college__  | (Superior incompleta)             |
#     | __11th__          | (3ro medio)                       |
#     | __HS-grad__       | (Secundaria completa)             |
#     | __Prof-school__   | (Escuela profesional)             |
#     | __ssoc-acdm__     | (Técnico superior administrativo) |
#     | __Assoc-voc__     | (Técnico superior vocacional)     |
#     | __9th__           | (1ro medio)                       |
#     | __7th-8th__       | (7mo-8vo)                         |
#     | __12th__          | (4to medio)                       |
#     | __Masters__       | (Maestría de postgrado)           |
#     | __1st-4th__       | (1ro-4to básico)                  |
#     | __10th__          | (2do medio)                       |
#     | __Doctorate__     | (Doctorado)                       |
#     | __5th-6th__       | (5to-6to)                         |
#     | __Preschool__     | (Preescolar).                     |
     
#     <br>  
    
# * `capital-gains`: Ingresos generados por inversiones fuera del trabajo asalariado
# * `capital-losses`: Pérdidas generadas por inversiones fuera del trabajo asalariado.
# * `fnlwgt`: Ponderador muestral.
# * `marital-status`: Estado civil del individuo: 

In [ ]:
# Mapa
gdf = gpd.GeoDataFrame(proc_df, geometry=gpd.points_from_xy(proc_df.xcoord, proc_df.ycoord))
df = gpd.read_file(gpd.datasets.get_path("nybb"))
df_wm = df.to_crs(epsg=3857)
ax = df.plot(figsize=(15,10), alpha=0.5, edgecolor = "k")
cx.add_basemap(ax,crs=df.crs, zoom =11, source=cx.providers.Stamen.TonerLite)
#cx.add_basemap(ax, source=cx.providers.Stamen.TonerLabels)
gdf.plot(ax=ax, color= "red", edgecolor = "black", markersize = 4)


#Incorporar al código de arriba para poder plotear variable objetivo:
# fig,ax = plt.subplots(1, 1) #plt.figure(figsize=(10,8))
# fig.set_size_inches(25, 25)
# df2009map[df2009map.arstmade == "N"].plot(color="red", ax=ax, markersize = 1)
# df2009map[df2009map.arstmade == "Y"].plot(ax=ax,  markersize = 12)
# plt.show()

In [ ]:
# función transformar x,y to lon, lat:
def xy_to_latlon(x,y):
    source_crs = 'epsg:2263' # Coordinate system of the file
    target_crs = 'epsg:4326' # Global lat-lon coordinate system
    polar_to_latlon = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)
    lat, lon = polar_to_latlon.transform(x,y)
    return lon, lat

In [ ]:
#otra opción de crear mapa con folium:
import folium
def generateBaseMap(loc, zoom=11, tiles='OpenStreetMap', crs='ESPG3857'):
    return folium.Map(location=loc,
                   #control_scale=True, 
                   zoom_start=zoom,
                   #tiles=tiles)
    )
base_map = generateBaseMap([40.7127837, -74.0059413])


marker = list(range(len(df2009.xcoord)))
counter = 0
tooltip = "Click Here For More Info"
icon = folium.features.CustomIcon('https://cdn-icons-png.flaticon.com/128/7500/7500224.png', icon_size=(40, 40))

for x,y in zip(df2009.xcoord, df2009.ycoord):
    lon, lat = xy_to_latlon(x,y)
    marker[counter] = folium.Marker(icon=icon,
    #     #location=[40.7127837, -74.0059413],
    location=[lon, lat],
    #     #popup="<stong>Allianz Arena</stong>",
    #     #tooltip=tooltip
    )
    marker[counter].add_to(base_map)
    #print(f"latitud {lat} y longitud {lon}")
    if counter>5 : 
        break
    counter += 1
base_map

In [ ]:
#Conteo de nulos
#proc_df.info(verbose=True, show_counts=True,memory_usage=False)

In [ ]:
#Descripción de variables númericas:
proc_df.describe().T

In [ ]:
#Descripción de variables categóricas: 
proc_df.describe(include="O").T


In [ ]:
#cargar df2010
# df2 = pd.read_csv('2010_1perc.csv', index_col='Unnamed: 0')
# df2.head()